In [11]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-04-30 11:12:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-04-30 11:12:31 (23.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [12]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [14]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2022-04-30 11:14:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.202.112, 142.250.79.112, 172.217.173.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.202.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   315KB/s    in 1.9s    

2022-04-30 11:14:59 (315 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [1]:
import pandas as pd
train_df = pd.read_csv('./datasets/nlp_getting_started/train.csv')
test_df = pd.read_csv('./datasets/nlp_getting_started/test.csv')

In [2]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_data_shuffled = train_df.sample(frac=1, random_state=42)
train_data_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [4]:
train_data_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [5]:
# splitting data into training and validation
from sklearn.model_selection import train_test_split

In [6]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_data_shuffled['text'].to_numpy(),
    train_data_shuffled['target'].to_numpy(),
    test_size=0.1,
    random_state=42)

In [7]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [8]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [9]:
text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    # pad_to_max_tokens=True
                                    )

In [10]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [11]:
text_vectorizer.adapt(train_sentences)

In [12]:
sample_sentence = 'How much does a bubble weigh'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  62,  187,  350,    3, 6056, 6857,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [13]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [14]:
len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [20]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length)
embedding

In [21]:
sample_sentence = 'How much does a bubble weigh'
sample_embed = embedding(text_vectorizer([sample_sentence]))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01728458, -0.02984915,  0.02142916, ...,  0.01065133,
         -0.00416965,  0.04388684],
        [-0.04443798,  0.03132567,  0.01163784, ..., -0.00419267,
          0.03019389, -0.03357557],
        [-0.01308882, -0.04107734,  0.01268766, ..., -0.04132646,
          0.04619626,  0.04237422],
        ...,
        [ 0.00576425, -0.04465927, -0.0130731 , ...,  0.01203073,
         -0.02691784, -0.01960808],
        [ 0.00576425, -0.04465927, -0.0130731 , ...,  0.01203073,
         -0.02691784, -0.01960808],
        [ 0.00576425, -0.04465927, -0.0130731 , ...,  0.01203073,
         -0.02691784, -0.01960808]]], dtype=float32)>

In [19]:
sample_embed.shape

TensorShape([1, 15, 128])